In [1]:
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
df = pd.read_csv('Skinpro - Skinpro.csv')
df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_1064\1753574390.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Skin type,Product,Concern,product_url,product_pic
0,Dry,Venusia Max Intensive Moisturizing Cream,"Acne,Hydration",https://dermatics.in/products/venusia-max-inte...,https://dermatics.in/cdn/shop/files/VENUSIA_4-...
1,"Oily,Combination",Saslic Foaming Face Wash,Whitehead/Blackhead,https://dermatics.in/products/cipla-saslic-foa...,https://dermatics.in/cdn/shop/files/saslic-foa...
2,Dry,The Derma Co 1% Hyaluronic Sunscreen Aqua Ultr...,Sun protection,https://amzn.to/464QBwM,https://m.media-amazon.com/images/I/51x3cj+-iU...
3,Dry,Hyalugel,Hydration,https://www.amazon.in/Hyalugel-Hyaluronic-Acid...,https://thebeautyinsideout.com/wp-content/uplo...
4,Normal,Himalaya Neem Face Wash,Pimples,https://himalayawellness.in/products/himalaya-...,https://himalayawellness.in/cdn/shop/files/100...


In [2]:
df.isnull().sum()

Skin type      0
Product        0
Concern        0
product_url    1
product_pic    0
dtype: int64

In [3]:
df.describe()

,Skin type,Product,Concern,product_url,product_pic
count,118,118,118,117,118
unique,29,118,34,117,118
top,Dry,Venusia Max Intensive Moisturizing Cream,Hydration,https://dermatics.in/products/venusia-max-inte...,https://dermatics.in/cdn/shop/files/VENUSIA_4-...
freq,22,1,19,1,1


In [4]:
df.dropna(inplace=True)

In [5]:
df.isnull().sum()

Skin type      0
Product        0
Concern        0
product_url    0
product_pic    0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [7]:
X = df['Product']
y = df['Skin type']  # Classification target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Support Vector Machine classifier
clf = SVC(kernel='linear')
clf.fit(X_train_tfidf, y_train)

# Evaluate the classifier
y_pred = clf.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

                                           precision    recall  f1-score   support

                                      Dry       0.36      1.00      0.53         4
                          Dry,Combination       0.00      0.00      0.00         1
                               Dry,Normal       0.00      0.00      0.00         1
    Dry,Oily,Normal,Combination,Sensitive       0.00      0.00      0.00         1
                            Dry,Sensitive       0.00      0.00      0.00         1
                                   Normal       0.00      0.00      0.00         2
Normal, Oily, Sensitive, Dry, Combination       0.00      0.00      0.00         1
    Normal,Dry,Combination,Oily,Sensitive       0.14      0.50      0.22         2
                                     Oily       0.00      0.00      0.00         2
                         Oily,Combination       0.00      0.00      0.00         5
                                Sensitive       0.00      0.00      0.00         1
   

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: Undefin

In [8]:
import IPython.display as display
def recommend_product(df, skin_type, concern):
    recommended_products = df[(df['Skin type'].str.contains(skin_type)) & (df['Concern'].str.contains(concern))]
    for index, row in recommended_products.iterrows():
        print("Product:", row['Product'])
        display.display(display.Image(url=row['product_pic']))


In [13]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd

app = Flask(__name__)
CORS(app, resources={r"/recommend": {"origins": ["http://localhost:8000", "http://localhost:5000/recommend"]}})

# Load the DataFrame once when the application starts
df = pd.read_csv('Skinpro - Skinpro.csv')

@app.route('/recommend', methods=['GET'])
def recommend_product():
    # Retrieve query parameters from the request
    skin_type = request.args.get('skin_type')  # corrected parameter name
    concern = request.args.get('concern')      # corrected parameter name
    
    # Check if both skin_type and concern are provided
    if skin_type is None or concern is None:
        return jsonify({'error': 'Both skin_type and concern parameters are required.'}), 400
    
    # Convert skin_type and concern to strings
    skin_type = str(skin_type)
    concern = str(concern)
    
    # Filter the DataFrame based on the query parameters
    recommended_products = df[(df['Skin type'].str.contains(skin_type, case=False)) &  (df['Concern'].str.contains(concern, case=False))]

    # Prepare the response data
    recommendations = []
    for index, row in recommended_products.iterrows():
        recommendations.append({
            'product_name': row['Product'],
            'product_image_url': row['product_pic']
        })
    
    # Return the recommendations as JSON
    return jsonify(recommendations)


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [17/Apr/2024 11:32:30] "GET / HTTP/1.1" 404 -


In [ ]:
# !pip install Flask pandas


1020


In [ ]:
#  pip install antlr4-tools

Note: you may need to restart the kernel to use updated packages.
